# Zero to Snowflake - Getting Started with Snowflake

## トピック
1. 仮想ウェアハウスと設定
2. 永続化クエリ結果の活用
3. 基本的なデータ変換技術
4. UNDROPによるデータ復旧
5. リソースモニター
6. 予算
7. ユニバーサル検索

## コンテキスト設定

In [ ]:
%%sql
USE DATABASE tb_101;
USE ROLE accountadmin;

## 1. 仮想ウェアハウスと設定

仮想ウェアハウスは、Snowflakeデータの分析を実行するために必要な動的でスケーラブル、かつコスト効率的な計算能力です。

In [ ]:
%%sql
-- 既存のウェアハウスを確認
SHOW WAREHOUSES;

In [ ]:
%%sql
-- ウェアハウスを作成
CREATE OR REPLACE WAREHOUSE my_wh
    COMMENT = 'My TastyBytes warehouse'
    WAREHOUSE_TYPE = 'standard'
    WAREHOUSE_SIZE = 'xsmall'
    MIN_CLUSTER_COUNT = 1
    MAX_CLUSTER_COUNT = 2
    SCALING_POLICY = 'standard'
    AUTO_SUSPEND = 60
    INITIALLY_SUSPENDED = true,
    AUTO_RESUME = false;

In [ ]:
%%sql
-- ウェアハウスを使用
USE WAREHOUSE my_wh;

In [ ]:
%%sql
-- ウェアハウスを再開
ALTER WAREHOUSE my_wh RESUME;
ALTER WAREHOUSE my_wh SET AUTO_RESUME = TRUE;

In [ ]:
%%sql
-- データをクエリ
SELECT * FROM raw_pos.truck_details;

## 2. ウェアハウススケーリング

In [ ]:
%%sql
ALTER WAREHOUSE my_wh SET warehouse_size = 'XLarge';

In [ ]:
%%sql
-- トラックごとの売上を確認
SELECT
    o.truck_brand_name,
    COUNT(DISTINCT o.order_id) AS order_count,
    SUM(o.price) AS total_sales
FROM analytics.orders_v o
GROUP BY o.truck_brand_name
ORDER BY total_sales DESC;

## 3. 永続化クエリ結果

クエリ結果は24時間キャッシュされます。

In [ ]:
%%sql
ALTER WAREHOUSE my_wh SET warehouse_size = 'XSmall';

## 4. 基本的なデータ変換

In [ ]:
%%sql
SELECT truck_build FROM raw_pos.truck_details;

In [ ]:
%%sql
-- ゼロコピークローニング
CREATE OR REPLACE TABLE raw_pos.truck_dev CLONE raw_pos.truck_details;

In [ ]:
%%sql
SELECT TOP 15 * 
FROM raw_pos.truck_dev
ORDER BY truck_id;

In [ ]:
%%sql
-- 新しい列を追加
ALTER TABLE raw_pos.truck_dev ADD COLUMN IF NOT EXISTS year NUMBER;
ALTER TABLE raw_pos.truck_dev ADD COLUMN IF NOT EXISTS make VARCHAR(255);
ALTER TABLE raw_pos.truck_dev ADD COLUMN IF NOT EXISTS model VARCHAR(255);

In [ ]:
%%sql
-- データを抽出して列を更新
UPDATE raw_pos.truck_dev
SET 
    year = truck_build:year::NUMBER,
    make = truck_build:make::VARCHAR,
    model = truck_build:model::VARCHAR;

In [ ]:
%%sql
SELECT year, make, model FROM raw_pos.truck_dev;

## 5. リソースモニター

In [ ]:
%%sql
USE ROLE accountadmin;

CREATE OR REPLACE RESOURCE MONITOR my_resource_monitor
    WITH CREDIT_QUOTA = 100
    FREQUENCY = MONTHLY
    START_TIMESTAMP = IMMEDIATELY
    TRIGGERS ON 75 PERCENT DO NOTIFY
             ON 90 PERCENT DO SUSPEND
             ON 100 PERCENT DO SUSPEND_IMMEDIATE;

ALTER WAREHOUSE my_wh 
    SET RESOURCE_MONITOR = my_resource_monitor;

## クリーンアップ

In [ ]:
%%sql
DROP RESOURCE MONITOR IF EXISTS my_resource_monitor;
DROP TABLE IF EXISTS raw_pos.truck_dev;
DROP WAREHOUSE IF EXISTS my_wh;